<a href="https://colab.research.google.com/github/betegon/16-QAM-Detection-MIMO/blob/master/TAC_MIMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import math
import numpy as np


## EJEMPLO CON 2 antenas


In [0]:
k = 2 # 2 antennas

In [79]:
s_matrix = np.array([[1, 1],[1, 1]])
print(s_matrix)
s = (np.array([s_matrix.flatten('F')])).transpose()
print("s\n",s)
t = s**2
print(t)

[[1 1]
 [1 1]]
s
 [[1]
 [1]
 [1]
 [1]]
[[1]
 [1]
 [1]
 [1]]


In [80]:
# CREACIÓN DE H 

k = 2  # number of antennas
# H is CN(0,1)
mu = np.zeros(2*k)
sigma = np.ones((k,k*2))


H = (1/np.sqrt(2))*np.random.normal(loc=mu, scale=sigma, size=(k,k*2)).view(np.complex128) 
print(H)
a = np.column_stack((np.real(H), -np.imag(H)))
print("Fila1: \n",a,"\n")
b = np.column_stack((np.imag(H), np.real(H)))
print("Fila2\n",b,"\n")
H_expanded = np.concatenate((a,b), axis=0)
print("Matriz H expandida\n",H_expanded)

[[ 1.14858562-0.43257711j -0.37347383-0.75870339j]
 [ 0.6119356 -1.62743362j  1.23376823-0.53825456j]]
Fila1: 
 [[ 1.14858562 -0.37347383  0.43257711  0.75870339]
 [ 0.6119356   1.23376823  1.62743362  0.53825456]] 

Fila2
 [[-0.43257711 -0.75870339  1.14858562 -0.37347383]
 [-1.62743362 -0.53825456  0.6119356   1.23376823]] 

Matriz H expandida
 [[ 1.14858562 -0.37347383  0.43257711  0.75870339]
 [ 0.6119356   1.23376823  1.62743362  0.53825456]
 [-0.43257711 -0.75870339  1.14858562 -0.37347383]
 [-1.62743362 -0.53825456  0.6119356   1.23376823]]


In [0]:
def awgn(signal, snr, seed=False):
    """ Generate Aditive White Gaussian Noise (AWGN) from an input signal.

    Args:
        signal  (np.array): Input signal. e.g. signal = np.array([[1, 2.3, -1]])
        snr        (float): Signal-to-noise ratio in dB.
        seed        (bool): Set seed to reproduce results (good for debugging).
                            Defaults to False.

    Returns:
        noise (np.array): AWGN generated from an input signal.

    """
    if seed: np.random.seed(1)
    sigpower = sum([ math.pow(abs(signal[i]),2) for i in range(len(signal)) ])
    sigpower = sigpower / len(signal)
    noisepower = sigpower / (math.pow(10,snr/10))
    noise = np.random.normal(0, np.sqrt(noisepower), len(signal))
    return np.array([noise]).T

In [82]:
# y = H*s+w;      
mu = np.zeros(k)
sigma = np.ones((k,k))
snr = 20

noise = awgn(s,snr)
# noise = np.random.normal(loc=mu, scale=sigma, size=(k,k)).view(np.complex128) 
# noise = np.concatenate((np.real(noise), np.imag(noise)), axis=0)



y = H_expanded.dot(s) + noise
print("\nH\n",H_expanded)
print("\ns\n",s)
# print("\nnoise\n",noise)
print("\nnoise \n", noise)
print("\ny\n",y)


H
 [[ 1.14858562 -0.37347383  0.43257711  0.75870339]
 [ 0.6119356   1.23376823  1.62743362  0.53825456]
 [-0.43257711 -0.75870339  1.14858562 -0.37347383]
 [-1.62743362 -0.53825456  0.6119356   1.23376823]]

s
 [[1]
 [1]
 [1]
 [1]]

noise 
 [[ 0.03190391]
 [-0.02493704]
 [ 0.14621079]
 [-0.20601407]]

y
 [[ 1.9982962 ]
 [ 3.98645498]
 [-0.26995791]
 [-0.52599842]]


In [83]:
## CALCULO DE W (variable a minimizar)
w = np.concatenate((s,t,[[1]]))
w_transpose = w.transpose()
print("w\n",w)
print("w_transpose\n",w_transpose)

W = w.dot(w_transpose)
print("\nBig W\n",W)
print("\nBig W\n",W.shape)

w
 [[1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]]
w_transpose
 [[1 1 1 1 1 1 1 1 1]]

Big W
 [[1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1]]

Big W
 (9, 9)


In [84]:
y


array([[ 1.9982962 ],
       [ 3.98645498],
       [-0.26995791],
       [-0.52599842]])

In [85]:
# Cálculo de la matriz a multiplicar por H

A11 = H_expanded.T.dot(H_expanded) # Check if it is really doing the transpose of H_expanded
A12 = np.zeros((2*k,2*k))
A13 = -H_expanded.T.dot(y) # Check if it is really doing the transpose of H_expanded
A1X = np.column_stack((np.column_stack((A11, A12)), A13))
print("\nFILA 1")
print("\n",A1X)

print("\nFILA 2")
A2X = np.zeros((2*k,A1X.shape[1]))
print("\n",A2X)

print("\nFILA 3")
A31 = (-y.T).dot(H_expanded)
A32= np.zeros((1,2*k))
A33 = (y.T).dot(y)
A3X = np.column_stack((np.column_stack((A31, A32)), A33))
A3X = np.column_stack((A31, A32, A33))
print("\n",A3X)

A = np.concatenate((A1X,A2X,A3X))
print("\n Matriz A")
print("\n",A)
print("\n",A.shape)

# A = [H'*H zeros(2*N,2*N) -H'*y; zeros(2*N,4*N+1); -y'*H zeros(1,2*N) y'*y];



FILA 1

 [[ 4.52937725  1.53019133  0.         -0.64550674  0.          0.
   0.          0.         -5.70747314]
 [ 1.53019133  2.52701555  0.64550674  0.          0.          0.
   0.          0.         -4.6599892 ]
 [ 0.          0.64550674  4.52937725  1.53019133  0.          0.
   0.          0.         -6.72016111]
 [-0.64550674  0.          1.53019133  2.52701555  0.          0.
   0.          0.         -3.11370375]]

FILA 2

 [[0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]]

FILA 3

 [[-5.70747314 -4.6599892  -6.72016111 -3.11370375  0.          0.
   0.          0.         20.2345626 ]]

 Matriz A

 [[ 4.52937725  1.53019133  0.         -0.64550674  0.          0.
   0.          0.         -5.70747314]
 [ 1.53019133  2.52701555  0.64550674  0.          0.          0.
   0.          0.         -4.6599892 ]
 [ 0.          0.64550674  4.52937725  1.53019133  0.          0.
   0.          0.         -6.7201

In [86]:
import cvxpy as cp
# Remove exponential notation
# np.set_printoptions(suppress=True)


# Problem data.
np.random.seed(1)
# print(A)
k = 2 # number of antennas (TX y RX)
zero_array = np.zeros(2*k)
# m = 2
# n = 2
# A = np.random.randn(m, n)
# b = np.random.randn(n,1)
#  2x2 2x1 = 2x1
# c = cp.matmul(A,b)
# print(c)

# Construct the problem.
W = cp.Variable((9,9), PSD=True)

objective = cp.Minimize(cp.trace(cp.matmul(W,A)))
# objective = cp.Minimize(cp.trace(W@A))
constraints = [W >> 0, 
               cp.diag(W[0:2*k, 0:2*k]) - W[2*k:4*k,4*k] == 0,
               cp.diag(W[2*k:4*k, 2*k:4*k]) - 10*W[2*k:4*k,4*k] + 9*np.ones((2*k)) == 0,
               W[4*k, 4*k] == 1 ]

'''
subject to:
1.  W >> 0 (semidefinida positiva)
2. diag(W(1:2*N,1:2*N)) - W(2*N+1:4*N,4*N+1) == 0
'''

prob = cp.Problem(objective, constraints)
prob.solve()

# print("Optimal value", prob.solve())
print("Optimal value of W")
print(W.value) # A numpy ndarray.

Optimal value of W
[[1.13329431 1.0645696  1.06844226 1.06455346 1.20644202 1.06454351
  1.07231171 1.06455183 1.06457474]
 [1.0645696  1.00000685 1.00364854 0.99999629 1.13328661 0.99998554
  1.00728684 0.99999698 1.00001618]
 [1.06844226 1.00364854 1.00729724 1.0036337  1.13739849 1.00362272
  1.01094822 1.00363423 1.00365337]
 [1.06455346 0.99999629 1.0036337  0.99998703 1.1332634  0.99997218
  1.00726788 0.99997731 1.00000141]
 [1.20644202 1.13328661 1.13739849 1.1332634  2.33285146 1.13271538
  1.31440019 1.13433884 1.13328932]
 [1.06454351 0.99998554 1.00362272 0.99997218 1.13271538 0.99997138
  1.00730387 0.99996193 0.99999645]
 [1.07231171 1.00728684 1.01094822 1.00726788 1.31440019 1.00730387
  1.07289259 1.0073996  1.00728998]
 [1.06455183 0.99999698 1.00363423 0.99997731 1.13433884 0.99996193
  1.0073996  0.99997904 0.99999873]
 [1.06457474 1.00001618 1.00365337 1.00000141 1.13328932 0.99999645
  1.00728998 0.99999873 1.0000173 ]]


In [87]:
W11=W.value[0:2*k,0:2*k]
print("Esto es W11:",W11)
print("\n")
W13=W.value[0:2*k,4*k]
print("Estos es W13:",W13)
print("\n")
W31=W.value[4*k,0:2*k]
print("Estos es W31:",W31)
print("\n")
W_1113=np.column_stack((W11,W13))
print("Column stack",W_1113)
one=np.append(W31,1)
one = np.array([one])
print("\n")
print(one)
print("\n")
print(W_1113.shape)
print(one.shape)
W_ED = np.concatenate((W_1113, one))
print("W_ED\n",W_ED)

Esto es W11: [[1.13329431 1.0645696  1.06844226 1.06455346]
 [1.0645696  1.00000685 1.00364854 0.99999629]
 [1.06844226 1.00364854 1.00729724 1.0036337 ]
 [1.06455346 0.99999629 1.0036337  0.99998703]]


Estos es W13: [1.06457474 1.00001618 1.00365337 1.00000141]


Estos es W31: [1.06457474 1.00001618 1.00365337 1.00000141]


Column stack [[1.13329431 1.0645696  1.06844226 1.06455346 1.06457474]
 [1.0645696  1.00000685 1.00364854 0.99999629 1.00001618]
 [1.06844226 1.00364854 1.00729724 1.0036337  1.00365337]
 [1.06455346 0.99999629 1.0036337  0.99998703 1.00000141]]


[[1.06457474 1.00001618 1.00365337 1.00000141 1.        ]]


(4, 5)
(1, 5)
W_ED
 [[1.13329431 1.0645696  1.06844226 1.06455346 1.06457474]
 [1.0645696  1.00000685 1.00364854 0.99999629 1.00001618]
 [1.06844226 1.00364854 1.00729724 1.0036337  1.00365337]
 [1.06455346 0.99999629 1.0036337  0.99998703 1.00000141]
 [1.06457474 1.00001618 1.00365337 1.00000141 1.        ]]


In [0]:
#quantiz version BT
def quantiz(entry, symbols):
    result = np.empty((len(entry),1))
    for i in range(len(entry)):
        minimum = float("inf")
        for val in symbols:
            if abs(val - entry[i]) < minimum:
                result[i,0] = val
                minimum = abs(val - entry[i])
    return result

In [89]:
#simple quantization 

valores=W.value[0:2*k,4*k]
print(quantiz(valores,s))

[[1.]
 [1.]
 [1.]
 [1.]]


In [90]:
#eigenvalue descomposition

u,s,v=np.linalg.svd(W_ED)#este te calcula svd directamente
print("Autovalores",s)
print ("\n")
print (W_ED)
N=2
print ("\n")
print(v)
print ("\n")
print(v[0,0:2*N])
print ("\n")
print(v[0,2*N])
eigen=v[0,0:2*N]/v[0,2*N]
eigen_trans=eigen.transpose()
print("\n")
print("ESto es el cociente:",eigen)
print("\n")
simb=[-3,-1, 1 ,3]
quantiz(eigen_trans,simb)

Autovalores [5.14060412 0.00001715 0.00000401 0.00000337 0.00000218]


[[1.13329431 1.0645696  1.06844226 1.06455346 1.06457474]
 [1.0645696  1.00000685 1.00364854 0.99999629 1.00001618]
 [1.06844226 1.00364854 1.00729724 1.0036337  1.00365337]
 [1.06455346 0.99999629 1.0036337  0.99998703 1.00000141]
 [1.06457474 1.00001618 1.00365337 1.00000141 1.        ]]


[[-0.46953138 -0.44105746 -0.44266198 -0.44105174 -0.4410582 ]
 [-0.22653376 -0.27649798 -0.21389266 -0.16220001  0.89452356]
 [-0.35760048 -0.17846647 -0.26544564  0.87594151 -0.05036575]
 [-0.35006395  0.83274817 -0.41479891 -0.09594753  0.0521694 ]
 [ 0.69123069 -0.06080607 -0.7181788   0.05181761 -0.00607478]]


[-0.46953138 -0.44105746 -0.44266198 -0.44105174]


-0.44105819916975014


ESto es el cociente: [1.06455652 0.99999833 1.0036362  0.99998536]




array([[1.],
       [1.],
       [1.],
       [1.]])

In [91]:
from numpy import linalg as la

def nearestPD(A):
    """Find the nearest positive-definite matrix to input
    A Python/Numpy port of John D'Errico's `nearestSPD` MATLAB code [1], which
    credits [2].
    [1] https://www.mathworks.com/matlabcentral/fileexchange/42885-nearestspd
    [2] N.J. Higham, "Computing a nearest symmetric positive semidefinite
    matrix" (1988): https://doi.org/10.1016/0024-3795(88)90223-6
    """

    B = (A + A.T) / 2
    _, s, V = la.svd(B)

    H = np.dot(V.T, np.dot(np.diag(s), V))

    A2 = (B + H) / 2

    A3 = (A2 + A2.T) / 2

    if isPD(A3):
        return A3

    spacing = np.spacing(la.norm(A))
    # The above is different from [1]. It appears that MATLAB's `chol` Cholesky
    # decomposition will accept matrixes with exactly 0-eigenvalue, whereas
    # Numpy's will not. So where [1] uses `eps(mineig)` (where `eps` is Matlab
    # for `np.spacing`), we use the above definition. CAVEAT: our `spacing`
    # will be much larger than [1]'s `eps(mineig)`, since `mineig` is usually on
    # the order of 1e-16, and `eps(1e-16)` is on the order of 1e-34, whereas
    # `spacing` will, for Gaussian random matrixes of small dimension, be on
    # othe order of 1e-16. In practice, both ways converge, as the unit test
    # below suggests.
    I = np.eye(A.shape[0])
    k = 1
    while not isPD(A3):
        mineig = np.min(np.real(la.eigvals(A3)))
        A3 += I * (-mineig * k**2 + spacing)
        k += 1

    return A3

def isPD(B):
    """Returns true when input is positive-definite, via Cholesky"""
    try:
        _ = la.cholesky(B)
        return True
    except la.LinAlgError:
        return False

if __name__ == '__main__':
    import numpy as np
    for i in range(10):
        for j in range(2, 100):
            A = np.random.randn(j, j)
            B = nearestPD(A)
            assert(isPD(B))
    print('unit test passed!')

unit test passed!


In [92]:
#randomization 
#from scipy.linalg import cholesky
#v2=cholesky(W_ED)
#v=np.linalg.cholesky(W_ED)
try:
    v= np.linalg.cholesky(W_ED)
    print("Es positiva")
except np.linalg.LinAlgError:
    v = nearestPD(W_ED)
    print("No Es positiva")
V=v.transpose()
print(V)
print("\n")
r=np.random.random((2*N+1,1))
print("Esto es r:",r)
Atransponer=V[:,0:2*N]
a=Atransponer.transpose()
print("\n")
cosas=np.dot(a,r) 
print("\n")
Btransponer=V[:,2*N]
b=Btransponer.transpose()
cosas1=np.dot(b,r)
print("\n")
Aquantiz=(cosas)/(cosas1)
print("Esto es Aquantiz:",Aquantiz)
print("\n")
randomization=quantiz(Aquantiz,simb)
print(randomization)

No Es positiva
[[1.13329664 1.0645696  1.0684425  1.06455428 1.0645712 ]
 [1.0645696  1.0000105  1.00364849 0.99999678 1.00001209]
 [1.0684425  1.00364849 1.00729973 1.00363435 1.00365003]
 [1.06455428 0.99999678 1.00363435 0.99998752 0.99999891]
 [1.0645712  1.00001209 1.00365003 0.99999891 1.00001373]]


Esto es r: [[0.61982542]
 [0.72161322]
 [0.82336413]
 [0.90780977]
 [0.00391546]]






Esto es Aquantiz: [[1.06455643]
 [0.99999829]
 [1.00363614]
 [0.99998563]]


[[1.]
 [1.]
 [1.]
 [1.]]
